In [1]:
#Author AJ, numpy import for ease of use
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix

In [2]:
data = pd.read_csv('train.csv', sep=',',header=0)
data = data.replace('negative',0).replace('positive',1)
cVec = CountVectorizer(binary=True,min_df=0.01)
features = cVec.fit_transform(data['review'].tolist())
print(cVec.get_feature_names())
print(features.toarray())


['10', '100', '15', '20', '30', '50', '70', '80', '90', 'ability', 'able', 'about', 'above', 'absolute', 'absolutely', 'accent', 'across', 'act', 'acted', 'acting', 'action', 'actor', 'actors', 'actress', 'actresses', 'acts', 'actual', 'actually', 'adaptation', 'add', 'added', 'adds', 'admit', 'adult', 'adventure', 'affair', 'after', 'again', 'against', 'age', 'ago', 'agree', 'ahead', 'air', 'alien', 'alive', 'all', 'allowed', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'amazing', 'america', 'american', 'americans', 'among', 'amount', 'amusing', 'an', 'and', 'animated', 'animation', 'annoying', 'another', 'answer', 'anti', 'any', 'anymore', 'anyone', 'anything', 'anyway', 'apart', 'apparently', 'appeal', 'appear', 'appearance', 'appears', 'appreciate', 'approach', 'are', 'aren', 'army', 'around', 'art', 'as', 'aside', 'ask', 'aspect', 'aspects', 'at', 'atmosphere', 'attack', 'attempt', 'attempts', 'attention', 'audience', 'audiences', 'available', 'averag